In [1]:
# Handling imports

import numpy as np
np.random.seed(1337) # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import MaxPooling2D, Conv2D
from keras.utils import np_utils
from keras import backend as K

import os
import glob
from skimage import io, transform, color, exposure

import pandas as pd

# Setting parameters for the network
batch_size = 128
nb_classes = 43
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 32, 32

# number of convolutional filters to use 
conv_depth_1 = 32
conv_depth_2 = 64
conv_depth_3 = 128
hidden_size = 512
# size of pooling area for max pooling 
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

Using TensorFlow backend.


In [2]:
# Making the images more uniform

def process_img(img):
    img = transform.resize(img, (img_rows, img_cols), mode='constant')
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)
    return img

In [3]:
# Load training images and labels

def get_class(img_path):
    return int(img_path.split('/')[-2])

train_dir = 'GTSRB-2/Final_Training/Images/'
train_images_paths = glob.glob(train_dir + '*/*.ppm', recursive=True)
np.random.shuffle(train_images_paths)
train_images = []
train_labels = []

for filename in train_images_paths:
    try:
        #img = preprocess_img(io.imread(filename))
        img = process_img(io.imread(filename))
        train_images.append(img)
        label = get_class(filename)
        train_labels.append(label)
    except (IOError, OSError):
        print('missed', filename)
        pass
    
X_train = np.array(train_images, dtype='float32')
# Make one hot targets
Y_train = np.eye(nb_classes, dtype='uint8')[train_labels] 

In [4]:
# Load test images and labels

test_dir = 'GTSRB/Final_Test/Images/'
test_images_paths = glob.glob(test_dir + '*.ppm', recursive=True)
np.random.shuffle(test_images_paths)
test_images = []
test_csv = pd.read_csv('GT-final_test.csv',sep=';')
test_labels = list(test_csv['ClassId'])

for filename in test_images_paths:
    try:
        #img = preprocess_img(io.imread(filename))
        img = process_img(io.imread(filename))
        test_images.append(img)
    except (IOError, OSError):
        print('missed', filename)
        pass
    
X_test = np.array(test_images, dtype='float32')
# Make one hot targets
Y_test = np.eye(nb_classes, dtype='uint8')[test_labels] 

In [5]:
# Processing images for theano or tensorflow backend
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols) 
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols) 
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3) 
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3) 
    input_shape = (img_rows, img_cols, 3)
    
# Converting the input from bytes to floats
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 

# Normalising the input
X_train /= np.max(X_train)
X_test /= np.max(X_test)

print('X_train shape:', X_train.shape) 
print(X_train.shape[0], 'train samples')
print(len(Y_train), 'train labels')
print(X_test.shape[0], 'test samples')
print(len(Y_test), 'test labels')

# convert class vectors to binary class matrices
#Y_train = np_utils.to_categorical(y_train, nb_classes) #breaks stuff
#Y_test = np_utils.to_categorical(y_test, nb_classes) #breaks stuff

X_train shape: (39209, 32, 32, 3)
39209 train samples
39209 train labels
12630 test samples
12630 test labels


In [6]:
# Defining the network

model = Sequential()

model.add(Conv2D(conv_depth_1, (kernel_size[0], kernel_size[1]),
                 padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(conv_depth_1, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D(conv_depth_2, (kernel_size[0], kernel_size[1]), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(conv_depth_2, (kernel_size[0], kernel_size[1]), activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Conv2D(conv_depth_3, (kernel_size[0], kernel_size[1]), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(conv_depth_3, (kernel_size[0], kernel_size[1]), activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(hidden_size))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta',
              metrics=['accuracy'])

In [8]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, 
          verbose=1, validation_data=(X_test, Y_test))

Train on 39209 samples, validate on 12630 samples
Epoch 1/12
39209/39209 [==============================] - 334s - loss: 2.6780 - acc: 0.2615 - val_loss: 8.5443 - val_acc: 0.0411
Epoch 2/12
39209/39209 [==============================] - 384s - loss: 0.6187 - acc: 0.7993 - val_loss: 11.7405 - val_acc: 0.0386
Epoch 3/12
39209/39209 [==============================] - 442s - loss: 0.2761 - acc: 0.9092 - val_loss: 13.7080 - val_acc: 0.0386
Epoch 4/12
 3456/39209 [=>............................] - ETA: 341s - loss: 0.2365 - acc: 0.9227

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/roman/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-b0da2f4104cc>", line 2, in <module>
    verbose=1, validation_data=(X_test, Y_test))
  File "/Users/roman/anaconda/lib/python3.5/site-packages/keras/models.py", line 853, in fit
    initial_epoch=initial_epoch)
  File "/Users/roman/anaconda/lib/python3.5/site-packages/keras/engine/training.py", line 1486, in fit
    initial_epoch=initial_epoch)
  File "/Users/roman/anaconda/lib/python3.5/site-packages/keras/engine/training.py", line 1141, in _fit_loop
    outs = f(ins_batch)
  File "/Users/roman/anaconda/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2103, in __call__
    feed_dict=feed_dict)
  File "/Users/roman/anaconda/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 767, in run
    run_metadata_ptr)
  File "/Us

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0])
print('Test accuracy:', score[1])